#  Imports


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Activation, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.applications import Xception, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.metrics import f1_score


import seaborn as sns
import matplotlib.pyplot as plt
import os

# Function Definitions

In [2]:
def train_val_generators(train_dir, val_dir):
    train_gen = ImageDataGenerator(rescale=1/255., horizontal_flip=True, vertical_flip=True, zoom_range=0.2)
    train_generator = train_gen.flow_from_directory(directory=train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')

    val_gen = ImageDataGenerator(rescale=1/255.)
    val_generator = val_gen.flow_from_directory(directory=val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')

    return train_generator, val_generator


def train_model(model, train_generator, val_generator, epochs=15):
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

    history = model.fit(train_generator, validation_data=val_generator, epochs=epochs, callbacks=[callback])

    return history


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def train_val_generators_shear(train_dir, val_dir):
    train_datagen = ImageDataGenerator(
        rescale=1./255,  # Normalizar los valores de píxeles entre 0 y 1
        rotation_range=20,  # Rango de rotación aleatoria en grados
        width_shift_range=0.2,  # Rango de traslación horizontal aleatoria
        height_shift_range=0.2,  # Rango de traslación vertical aleatoria
        shear_range=0.2,  # Rango de cizallamiento aleatorio
        zoom_range=0.2,  # Rango de zoom aleatorio
        horizontal_flip=True,  # Realizar volteo horizontal aleatorio
        vertical_flip=True,  # Realizar volteo vertical aleatorio
        fill_mode='nearest'  # Estrategia de relleno para transformaciones
    )

    val_datagen = ImageDataGenerator(rescale=1./255.)

    train_generator = train_datagen.flow_from_directory(
        directory=train_dir,
        target_size=IMG_SIZE,  # Tamaño al que se redimensionarán las imágenes
        batch_size=BATCH_SIZE,  # Tamaño del lote de datos
        class_mode='categorical'  # Modo de clasificación para problemas de múltiples clases
    )

    val_generator = val_datagen.flow_from_directory(
        directory=val_dir,
        target_size=IMG_SIZE,  # Tamaño al que se redimensionarán las imágenes
        batch_size=BATCH_SIZE,  # Tamaño del lote de datos
        class_mode='categorical'  # Modo de clasificación para problemas de múltiples clases
    )

    return train_generator, val_generator


In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

import seaborn as sns
import matplotlib.pyplot as plt

def plot_training_history(history):
    # Plotear el historial de entrenamiento
    eps = range(len(history.history["loss"]))
    plt.figure(figsize=(10, 6))
    plt.plot(eps, history.history["loss"])
    plt.plot(eps, history.history["val_loss"])
    plt.legend(["loss", "val_loss"])
    plt.title('Training History - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()

    # Plotear la precisión
    plt.figure(figsize=(10, 6))
    plt.plot(eps, history.history["accuracy"])
    plt.plot(eps, history.history["val_accuracy"])
    plt.legend(["accuracy", "val_accuracy"])
    plt.title('Training History - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()

def check_precision(model, test_generator):
    # Asignar e imprimir etiquetas
    test_labels, pred_labels = assign_labels(test_generator, model, print_info=False)

    # Imprimir la matriz de confusión y la precisión
    cm = confusion_matrix(test_labels, pred_labels)
    accuracy = accuracy_score(test_labels, pred_labels)

    print("Matriz de Confusión:")
    print(cm)
    print("Precisión:", accuracy)
    return cm





In [5]:
def plot_confusion_matrix(cm, classes):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Matriz de Confusión')
    plt.xlabel('Etiquetas Predichas')
    plt.ylabel('Etiquetas Verdaderas')
    plt.show()

# Configuracion inicial

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:

# Directorio base después de montar el drive
base_path = '/content/drive/My Drive/DataSet'

# Listar archivos en el directorio base
files = os.listdir(base_path)

path_root = '/content/drive/My Drive/DataSet'

valid_dir = os.path.join(path_root, 'val')
train_dir = os.path.join(path_root, 'train')
test_dir = os.path.join(path_root, 'test')

In [8]:
bc_types = [file for file in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, file))]
print("Types: ", bc_types)

# to encode labels
bc_dict = {'InSitu': 3, 'Benign': 2, 'Normal': 1, 'Invasive': 4}
print("Encode: ", bc_dict)

# to decode labels
dict_bc = {1: "Normal", 2: "Benign", 3: "InSitu", 4: "Invasive"}
print("Decode: ", dict_bc)

classes = [dict_bc[i] for i in sorted(dict_bc.keys())]


Types:  ['InSitu', 'Benign', 'Invasive', 'Normal']
Encode:  {'InSitu': 3, 'Benign': 2, 'Normal': 1, 'Invasive': 4}
Decode:  {1: 'Normal', 2: 'Benign', 3: 'InSitu', 4: 'Invasive'}


In [9]:
# Set IMG_SIZE and BATCH_SIZE
IMG_SIZE = (512, 512)
BATCH_SIZE = 8
NUM_CLASES = 4


In [10]:
train_generator, val_generator = train_val_generators_shear(train_dir, valid_dir)   # create image generators to feed images to the model


Found 236 images belonging to 4 classes.
Found 81 images belonging to 4 classes.


In [11]:
train_generator_2, val_generator_2 = train_val_generators_shear(train_dir, valid_dir)   # create image generators to feed images to the model


Found 236 images belonging to 4 classes.
Found 81 images belonging to 4 classes.


# Create models

In [12]:
#Para poder incluir el valor f1 en el entrenamiento del modelo creamos las siguientes
#funciones para añadirlo como una función siguiendo la referencia del sigueinte enlace
#https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model

from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



## ENTRENAMIENTO DE MODELOS

In [13]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import Xception, EfficientNetB0, InceptionV3, VGG16, VGG19, ResNet50
from tensorflow.keras.optimizers import Adam
import tensorflow as tf


In [18]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models

def build_model_with_base_v2(base_model, num_classes, img_size1, img_size2, learning_rate, freeze_layers=None, additional_layers=None):
    inputs = layers.Input(shape=(img_size1, img_size2, 3))
    base_model = base_model(include_top=False, input_tensor=inputs)

    # Congelar todas las capas si freeze_layers es None
    if freeze_layers == 'all':
        for layer in base_model.layers:
            layer.trainable = False
    elif isinstance(freeze_layers, int):
        # Congelar las primeras 'freeze_layers' capas
        for layer in base_model.layers[:freeze_layers]:
            layer.trainable = False
    else:
        raise ValueError("El argumento freeze_layers debe ser 'all' o un entero.")

    # Construir el modelo con el modelo base

    x = base_model.output

    # Reducir el número de capas de convolución
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2, 2)(x)

    # Agregar una capa de convolución adicional
    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2, 2)(x)

    # Agregar GlobalAveragePooling2D
    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    # Compilar el modelo
    model = models.Model(inputs, outputs, name=f"CustomModel_{base_model.name}")
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
     loss="categorical_crossentropy",
                  metrics=['acc', f1_m, precision_m, recall_m])

    return model


In [20]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import RMSprop  # Cambiando a RMSprop
from tensorflow.keras import layers, models

def build_model_with_base_adam(base_model, num_classes, img_size1, img_size2, learning_rate, freeze_layers=None, additional_layers=None):
    inputs = layers.Input(shape=(img_size1, img_size2, 3))
    base_model = base_model(include_top=False, input_tensor=inputs)

    # Congelar todas las capas si freeze_layers es None
    if freeze_layers == 'all':
        for layer in base_model.layers:
            layer.trainable = False
    elif isinstance(freeze_layers, int):
        # Congelar las primeras 'freeze_layers' capas
        for layer in base_model.layers[:freeze_layers]:
            layer.trainable = False
    else:
        raise ValueError("El argumento freeze_layers debe ser 'all' o un entero.")

    # Construir el modelo con el modelo base

    x = base_model.output

    # Reducir el número de capas de convolución
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2, 2)(x)

    # Agregar una capa de convolución adicional
    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2, 2)(x)

    # Agregar GlobalAveragePooling2D
    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    # Compilar el modelo
    model = models.Model(inputs, outputs, name=f"CustomModel_{base_model.name}")
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss="categorical_crossentropy",
                  metrics=['acc', f1_m, precision_m, recall_m])

    return model

In [21]:
xception_model_adam = build_model_with_base_adam(Xception, NUM_CLASES, IMG_SIZE[0], IMG_SIZE[1],0.00001,0)


# Entrenamiento de modelos

In [22]:
# Specify the directory to save the models
save_dir = '/content/drive/My Drive/SavedModels'

# Ensure the directory exists, create it if necessary
os.makedirs(save_dir, exist_ok=True)

In [23]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [24]:
print("\n[INFO] Ready to train. Training is starting!\n")
BATCH_SIZE = 8



[INFO] Ready to train. Training is starting!



In [25]:
# Model training for Xception
%%time
#hist_xception_preprocessed = xception_model_preprocessed.fit(train_generator, validation_data=val_generator, epochs=50,callbacks=[callback])

# Guardar modelos en formato H5


CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.3 µs


In [26]:
# Model training for Xception
%%time

hist_xception_adam = xception_model_adam.fit(train_generator, validation_data=val_generator, epochs=50, callbacks=[callback])

# Guardar modelos en formato H5


Epoch 1/50
30/30 [==============================] - 507s 16s/step - loss: 1.4857 - acc: 0.3051 - f1_m: 0.2096 - precision_m: 0.3961 - recall_m: 0.1500 - val_loss: 1.4169 - val_acc: 0.2469 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/50
30/30 [==============================] - 24s 786ms/step - loss: 1.4023 - acc: 0.3686 - f1_m: 0.2688 - precision_m: 0.4383 - recall_m: 0.2042 - val_loss: 1.3830 - val_acc: 0.2840 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 3/50
30/30 [==============================] - 23s 762ms/step - loss: 1.2623 - acc: 0.4703 - f1_m: 0.3328 - precision_m: 0.5250 - recall_m: 0.2500 - val_loss: 1.3460 - val_acc: 0.3457 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 4/50
30/30 [==============================] - 24s 787ms/step - loss: 1.1569 - acc: 0.5339 - f1_m: 0.4248 - precision_m: 0.6617 - recall_m: 0.3250 - val_loss: 1.3015 - val_acc: 0.3704 - val

# Evaluacion de modelos

# validacion de resultados

In [32]:
from tensorflow.keras.models import load_model
import os

save_dir = '/content/drive/My Drive/SavedModels'


#xception_model_preprocessed.save(os.path.join(save_dir, 'final_xception_zoom_v1.h5'))
#xception_model_preprocessed_2.save(os.path.join(save_dir, 'final_xception_zoom_v2.h5'))
xception_model_adam.save(os.path.join(save_dir, 'xception_model_adam.h5'))
#xception_model_generator_4.save(os.path.join(save_dir, 'final_xception_zoom_v4.h5'))



/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [33]:
from tensorflow.keras.models import load_model

model_path = '/content/drive/My Drive/SavedModels/xception_model_adam.h5'

modelo_entrenado = load_model(model_path, custom_objects={'precision_m': precision_m, 'f1_m': f1_m,'recall_m': recall_m})


In [34]:
# Evaluación en el conjunto de prueba
test_gen = ImageDataGenerator(rescale=1/255.)
test_generator = test_gen.flow_from_directory(directory=test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')



Found 83 images belonging to 4 classes.


In [35]:
import pandas as pd
from sklearn.metrics import classification_report
from tabulate import tabulate

def evaluate_and_print_metrics(model, model_name, test_generator):
    # Evaluación en el conjunto de prueba
    evaluation = model.evaluate(test_generator)

    # Obtener las predicciones para calcular otras métricas
    predictions = model.predict(test_generator)

    # Calcular métricas adicionales usando classification_report de scikit-learn
    report = classification_report(test_generator.classes, predictions.argmax(axis=1), target_names=test_generator.class_indices)

    # Obtener las métricas de precisión para cada modelo
    precision = evaluation[1]  # Extraer las métricas de interés
    f1 = float(report.split()[-2])
    recall = float(report.split()[-4])


    return report


In [ ]:
result = evaluate_and_print_metrics(modelo_entrenado, 'xception_model_adam.h5', test_generator)


 4/11 [=========>....................] - ETA: 1:18 - loss: 0.4672 - acc: 0.8125 - f1_m: 0.7854 - precision_m: 0.8259 - recall_m: 0.7500

In [ ]:
result

In [ ]:
import matplotlib.pyplot as plt

def plot_hist(hist):
    # Crear la figura y los ejes
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plotear accuracy y val_accuracy
    ax.plot(hist.history["acc"], label="Train Accuracy", color="blue", linestyle="-")
    ax.plot(hist.history["val_acc"], label="Validation Accuracy", color="blue", linestyle="--")

    # Plotear precision_m y val_precision_m
    ax.plot(hist.history["precision_m"], label="Train Precision", color="green", linestyle="-")
    ax.plot(hist.history["val_precision_m"], label="Validation Precision", color="green", linestyle="--")

    # Configurar el subplot
    ax.set_title("Model Metrics")
    ax.set_ylabel("Metrics")
    ax.set_xlabel("Epoch")
    ax.legend(loc="upper left")

    # Mostrar la gráfica
    plt.show()

# Ejemplo de uso:
# Suponiendo que `hist` es el objeto devuelto por el entrenamiento del modelo
# model.fit(...)
# plot_hist(hist)


In [ ]:
import matplotlib.pyplot as plt

def plot_metrics_loss_f1(history, name):
    fig, ax = plt.subplots(figsize=(10, 6))
    fig.suptitle('Model Metrics Comparison', fontsize=16)

    # Pérdida
    ax.plot(history.history['loss'], label='Train Loss', color='blue', linestyle='-')
    ax.plot(history.history['val_loss'], label='Validation Loss', color='blue', linestyle='--')

    # F1 Score
    ax.plot(history.history['f1_m'], label='Train F1 Score', color='orange', linestyle='-')
    ax.plot(history.history['val_f1_m'], label='Validation F1 Score', color='orange', linestyle='--')

    ax.set_title(name)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Metrics')
    ax.legend()

    plt.show()

# Ejemplo de uso:
# Suponiendo que `history` y `name` son los datos proporcionados para un solo modelo
# plot_metrics_loss_f1(history, name)


In [ ]:
def plot_confusion_matrix(cm, classes):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Matriz de Confusión')
    plt.xlabel('Etiquetas Predichas')
    plt.ylabel('Etiquetas Verdaderas')
    plt.show()

In [ ]:
import matplotlib.pyplot as plt

def plot_hist(hist):
    # Crear la figura y los ejes
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plotear accuracy y val_accuracy
    ax.plot(hist.history["acc"], label="Train Accuracy", color="blue", linestyle="-")
    ax.plot(hist.history["val_acc"], label="Validation Accuracy", color="blue", linestyle="--")

    # Plotear precision_m y val_precision_m
    ax.plot(hist.history["precision_m"], label="Train Precision", color="green", linestyle="-")
    ax.plot(hist.history["val_precision_m"], label="Validation Precision", color="green", linestyle="--")

    # Configurar el subplot
    ax.set_title("Model Metrics")
    ax.set_ylabel("Metrics")
    ax.set_xlabel("Epoch")
    ax.legend(loc="upper left")

    # Mostrar la gráfica
    plt.show()

# Ejemplo de uso:
# Suponiendo que `hist` es el objeto devuelto por el entrenamiento del modelo
# model.fit(...)
# plot_hist(hist)


In [ ]:
plot_hist(hist_xception_preprocessed_2)

In [ ]:
plot_metrics_loss_f1(hist_xception_preprocessed_2,'hist_xception_preprocessed')

In [ ]:
def check_precision(model, test_generator):
    # Asignar e imprimir etiquetas
    test_labels, pred_labels = assign_labels(test_generator, model, print_info=False)

    # Imprimir la matriz de confusión y la precisión
    cm = confusion_matrix(test_labels, pred_labels)
    accuracy = accuracy_score(test_labels, pred_labels)

    print("Matriz de Confusión:")
    print(cm)
    print("Precisión:", accuracy)
    return cm

In [ ]:
def assign_labels(test_generator, model, print_info=True):
    test_labels = []
    pred_labels = []

    for batch in range(0, int(len(test_generator))):
        batch_predicted = model.predict(test_generator[batch][0])
        for i in range(len(batch_predicted)):
            pred_label = np.argmax(batch_predicted[i])
            true_label = np.argmax(test_generator[batch][1][i])
            true_name = bc_types[true_label]  # Decode true label to category
            pred_name = bc_types[pred_label]  # Decode predicted label to category
            true_category = dict_bc[bc_dict[true_name]]  # Use bc_dict to get the value and then access dict_bc
            pred_category = dict_bc[bc_dict[pred_name]]  # Use bc_dict to get the value and then access dict_bc

            pred_labels.append(pred_category)
            test_labels.append(true_category)

            if print_info:
                print(f"Imagen {batch * len(batch_predicted) + i + 1} - Verdadero: {true_category} ( {true_name} ), Predicho: {pred_category} ( {pred_name} )")

    return test_labels, pred_labels

In [ ]:
cm_xception = check_precision(modelo_entrenado,test_generator)

In [ ]:
plot_confusion_matrix(cm_xception, classes)


Area bajo la curca ROC

In [ ]:
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

# Supongamos que ya tienes las etiquetas reales y predichas
test_labels, pred_labels = assign_labels(test_generator, modelo_entrenado, print_info=False)



In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

# Convertir las etiquetas a formato binario
test_labels_numeric = [bc_dict[label] for label in test_labels]
pred_labels_numeric = [bc_dict[label] for label in pred_labels]

test_labels_bin = label_binarize(test_labels_numeric, classes=[1, 2, 3, 4])
pred_labels_bin = label_binarize(pred_labels_numeric, classes=[1, 2, 3, 4])

# Calcular la curva ROC para cada clase
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(NUM_CLASES):
    fpr[i], tpr[i], _ = roc_curve(test_labels_bin[:, i], pred_labels_bin[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plotear la curva ROC para cada clase
plt.figure(figsize=(12, 8))
colors = ['blue', 'orange', 'green', 'red']
classes_names = ['Normal', 'Benign', 'InSitu', 'Invasive']

for i, color, class_name in zip(range(NUM_CLASES), colors, classes_names):
    plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'{class_name} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC para Clasificación Multiclase')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Definir las clases, precisiones y recalls
classes = ['Normal', 'Benign', 'InSitu', 'Invasive']
precisions = [0.65, 0.83, 0.8, 0.89]
recalls = [0.81, 0.75, 0.76, 0.81]

# Precisión y recall globales
global_precision = 0.7831
global_recall = 0.7462

# Valor de referencia para la precisión
reference_precision = 0.79

# Crear la figura y los ejes
fig, ax = plt.subplots(figsize=(10, 6))

# Plotear las barras de precisión y recalls con colores suaves
bar_width = 0.35
bar_positions = range(len(classes))
bars_precision = ax.bar(bar_positions, precisions, width=bar_width, color='skyblue', label='Precision')
bars_recall = ax.bar([pos + bar_width for pos in bar_positions], recalls, width=bar_width, color='lightcoral', label='Recall')

# Plotear la línea de referencia para la precisión
ax.axhline(y=reference_precision, color='gray', linestyle='--', label='Precision del modelo (0.79)')

# Anotar los valores de precisión y recalls en las barras
for bar, precision, recall in zip(bars_precision, precisions, recalls):
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval, f'P: {round(precision, 2)}\nR: {round(recall, 2)}', ha='center', va='bottom')

# Etiquetas y título
ax.set_xticks([pos + bar_width/2 for pos in bar_positions])
ax.set_xticklabels(classes)
ax.set_ylabel('Score')
ax.set_title('Precision y Recall para cada Clase')

# Mostrar la gráfica
fig.text(0.7, 0.015, f'Precisión del modelo: {round(global_precision, 4)}\nRecall del modelo: {round(global_recall, 4)}', ha='center', va='center', color='darkgreen', fontsize=10)

# Ubicar la leyenda abajo a la derecha
ax.legend(loc="lower right")

plt.show()
